# 05. Visualization and Chart Generation

This notebook covers:
- Creating charts from query results
- Different types of visualizations
- Auto-visualization based on data
- Interactive charts with Plotly

In [ ]:
import sys
sys.path.append('/workspace')

from src.utils.db_utils import DatabaseConnection
from src.utils.text2sql_utils import execute_text2sql
from src.utils.viz_utils import (
    create_bar_chart,
    create_line_chart,
    create_pie_chart,
    create_scatter_plot,
    auto_visualize
)
import pandas as pd

print("✓ Libraries imported successfully")

In [ ]:
# Initialize database
db = DatabaseConnection()

## 1. Bar Chart - Sales by Region

In [ ]:
query = "What is the total sales amount by region?"
result = execute_text2sql(db, query)

if result['success']:
    df = result['results']
    fig = create_bar_chart(df, 'region', 'sum', title='Total Sales by Region')
    fig.show()
else:
    print(f"Error: {result['error']}")

## 2. Pie Chart - Employees by Department

In [ ]:
query = "How many employees are in each department?"
result = execute_text2sql(db, query)

if result['success']:
    df = result['results']
    # Assuming the result has department_name and count columns
    if len(df.columns) >= 2:
        fig = create_pie_chart(df, df.columns[0], df.columns[1], 
                              title='Employee Distribution by Department')
        fig.show()
else:
    print(f"Error: {result['error']}")

## 3. Scatter Plot - Salary Analysis

In [ ]:
query = "SELECT e.employee_id, e.salary, e.job_title, d.department_name FROM employees e JOIN departments d ON e.department_id = d.department_id"
df = db.execute_query_df(query)

fig = create_scatter_plot(df, 'employee_id', 'salary', color_col='department_name',
                         title='Employee Salary Distribution')
fig.show()

## 4. Time Series - Sales Over Time

In [ ]:
query = "SELECT sale_date, SUM(total_amount) as daily_total FROM sales GROUP BY sale_date ORDER BY sale_date"
df = db.execute_query_df(query)

# Convert sale_date to datetime
df['sale_date'] = pd.to_datetime(df['sale_date'])

fig = create_line_chart(df, 'sale_date', 'daily_total', 
                       title='Daily Sales Trend')
fig.show()

## 5. Auto Visualization

In [ ]:
# Let the system automatically choose the best visualization
queries = [
    "What is the average salary by department?",
    "Show me sales by category",
    "List all projects with their budgets"
]

for query in queries:
    print(f"\nQuery: {query}")
    result = execute_text2sql(db, query)
    
    if result['success']:
        viz = auto_visualize(result['results'], query)
        if viz is not None and hasattr(viz, 'show'):
            viz.show()
        else:
            display(viz)
    else:
        print(f"Error: {result['error']}")

## 6. Combined Query and Visualization Pipeline

In [ ]:
def query_and_visualize(natural_query: str):
    """Complete pipeline: natural language -> SQL -> results -> chart"""
    print(f"Query: {natural_query}")
    print("="*60)
    
    result = execute_text2sql(db, natural_query)
    
    if result['success']:
        print(f"\nGenerated SQL:\n{result['sql_query']}\n")
        print(f"Retrieved {result['row_count']} rows")
        
        # Show data
        print("\nData:")
        display(result['results'])
        
        # Auto-visualize
        print("\nVisualization:")
        viz = auto_visualize(result['results'], natural_query)
        if viz is not None and hasattr(viz, 'show'):
            viz.show()
    else:
        print(f"Error: {result['error']}")

# Test the pipeline
query_and_visualize("What is the total budget by department?")

## 7. Dashboard-Style Multiple Charts

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create multiple charts in one figure
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "xy"}, {"type": "domain"}],
           [{"type": "xy"}, {"type": "xy"}]],
    subplot_titles=('Sales by Region', 'Employees by Department', 
                    'Department Budgets', 'Salary Distribution')
)

# Chart 1: Sales by region
df1 = db.execute_query_df("SELECT region, SUM(total_amount) as total FROM sales GROUP BY region")
fig.add_trace(go.Bar(x=df1['region'], y=df1['total'], name='Sales'), row=1, col=1)

# Chart 2: Employees by department
df2 = db.execute_query_df("SELECT d.department_name, COUNT(*) as count FROM employees e JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name")
fig.add_trace(go.Pie(labels=df2['department_name'], values=df2['count'], name='Employees'), row=1, col=2)

# Chart 3: Department budgets
df3 = db.execute_query_df("SELECT department_name, budget FROM departments ORDER BY budget DESC")
fig.add_trace(go.Bar(x=df3['department_name'], y=df3['budget'], name='Budget'), row=2, col=1)

# Chart 4: Salary distribution
df4 = db.execute_query_df("SELECT salary FROM employees")
fig.add_trace(go.Histogram(x=df4['salary'], name='Salary'), row=2, col=2)

fig.update_layout(height=800, showlegend=False, title_text="Company Analytics Dashboard")
fig.show()

## Summary

In this notebook, you learned:
- ✓ How to create various types of charts (bar, pie, line, scatter)
- ✓ How to use auto-visualization
- ✓ How to build a complete query-to-chart pipeline
- ✓ How to create dashboard-style visualizations

Next: Move to `06_end_to_end.ipynb` for a complete end-to-end demonstration.